<a href="https://colab.research.google.com/github/nechebarrena/HOML/blob/main/HOML_cap_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CAPITULO 14 - HANDS ON MACHINE LEARNING**
## **Vision por computadora usando CNN's**

A lo largo de este capitulo el libro se propone analizar las redes neuronales convolucionales (CNN) y su implementacion con TensorFlow y Keras aplicadas a la vision por computadora. En capitulos posteriores va a utilizar tambien CNN's para otro tipo de tareas.

### **Capas convolucionales**
Las capas convolucionales son los ladrillos elementales para construir una CNN. 
Las neuronaes en estas capas no estan conectadas a todos los datos de entrada (que de ahora en mas vamos a suponer que es una matriz donde cada elemento es un pixel y por lo tanto representa una imagen) sino a un pequeño grupo llamado _receptive fields_. Esta forma de conectar las neuronas permite que la red se concentre en atributos de pequeña escala.

_ver lo de zero padding_

En general los campos de vision o _receptive fields_ entre dos neuronas contiguas se suelen solapar, tal como se muestra en la figura $14.3$ del libro, sin embargo el tamaño del solapamiento puede ser elegido segun la arquitectura. El parametro de control que nos permite seleccionar cuan solapado esta el campo de vision de dos neuronas contiguas se denomina _stride_. Aumentar el stride reduce el solapamiento y esto, obviamente, reduce la cantidad de neuronas en la capa. Esto se ejemplifica en la figura $14.4$ del libro.

_Aca el libro no explica demasiado como se conectan ni como son estas neuronas, quizas mas adelante lo explique mejor. Si no lo hace volver aca y explicarlo con alguna otra fuente_

### **Filtros**

Si pensamos los pesos de una neurona como una matriz que puede tomar distintos valores, poner cero en un elemento implica que no va a considerar el pixel del cual proviene. Por el contrario, poner un uno implica lo contrario, un transpaso perfecto del pixel original. De esta forma, si todas las neuronas en una capa tienen un unico "filtro" aplicado a sus pesos entonces la salida de esta imagen sera una nueva imagen donde se resalta las caracteristicas del filtro usado. Ejmeplos tipicos de filtros son los verticales, horizontales, de blurreo, etc. La idea es que uno no seleccione estos filtros sino que sea la red quien aprenda cuales son los filtros ideales dada una funcion de costo y un conjunto de entrenamiento. 

## **Apilando distintas capas de atributos**

Hasta el momento solo utilizamos un "filtro" o "seleccionador de atributo" por capa, pero de forma mas general uno puede utilizar mas de uno por capa. O sea, puede buscar muchos atributos en una misma capa. Cuando utilizamos un solo filtro por capa lo que estamos haciendo es utilizar una cantidad de $N \times M$ neuronas, donde este es el numero de pixeles en la capa. Todas estas neuronas comparten los mismos pesos y el mismo bias. Cuando utilizamos mas de un filtro por capa, cada uno de estos filtros comparte los pesos y bias para todas sus neuronas y a su vez estos difieren de los de las neuronas de los otros filtros. 

